# Document Ingestion & Chunking Pipeline

This notebook handles:
1. Ingesting ~10 documents from a local directory
2. Cleaning text (whitespace, special chars, encoding issues)
3. Chunking with `chunk_size=600`, `overlap=100`
4. Computing & logging statistics
5. Printing sample chunks

In [ ]:
# === Configuration ===
import os
import re
import json
import glob
import logging
from pathlib import Path
from datetime import datetime

CHUNK_SIZE = 600      # characters per chunk
CHUNK_OVERLAP = 100   # overlap between consecutive chunks
DOCS_DIR = "./documents"  # <-- UPDATE to your documents folder
SUPPORTED_EXTENSIONS = [".txt", ".md", ".pdf", ".docx", ".html", ".csv", ".json"]

# Logging setup
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[
        logging.FileHandler("chunking_pipeline.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)
logger.info("Pipeline started")

## 1. Document Ingestion

In [ ]:
def load_document(filepath: str) -> str:
    """Load a single document and return its raw text."""
    ext = Path(filepath).suffix.lower()
    
    if ext in (".txt", ".md", ".csv"):
        with open(filepath, "r", encoding="utf-8", errors="replace") as f:
            return f.read()
    
    elif ext == ".json":
        with open(filepath, "r", encoding="utf-8") as f:
            data = json.load(f)
        # Flatten JSON to text
        return json.dumps(data, indent=2, ensure_ascii=False)
    
    elif ext == ".html":
        try:
            from bs4 import BeautifulSoup
            with open(filepath, "r", encoding="utf-8", errors="replace") as f:
                soup = BeautifulSoup(f.read(), "html.parser")
            return soup.get_text(separator=" ")
        except ImportError:
            logger.warning("bs4 not installed; reading HTML as raw text")
            with open(filepath, "r", encoding="utf-8", errors="replace") as f:
                return f.read()
    
    elif ext == ".pdf":
        try:
            import fitz  # PyMuPDF
            doc = fitz.open(filepath)
            return "\n".join(page.get_text() for page in doc)
        except ImportError:
            logger.error("PyMuPDF not installed. Run: pip install pymupdf")
            return ""
    
    elif ext == ".docx":
        try:
            from docx import Document
            doc = Document(filepath)
            return "\n".join(p.text for p in doc.paragraphs)
        except ImportError:
            logger.error("python-docx not installed. Run: pip install python-docx")
            return ""
    
    else:
        logger.warning(f"Unsupported file type: {ext} — skipping {filepath}")
        return ""


def ingest_documents(docs_dir: str) -> dict[str, str]:
    """Scan directory and load all supported documents."""
    documents = {}
    for ext in SUPPORTED_EXTENSIONS:
        for fpath in glob.glob(os.path.join(docs_dir, f"*{ext}")):
            name = os.path.basename(fpath)
            text = load_document(fpath)
            if text.strip():
                documents[name] = text
                logger.info(f"Loaded: {name} ({len(text):,} chars)")
            else:
                logger.warning(f"Empty or unreadable: {name}")
    logger.info(f"Total documents ingested: {len(documents)}")
    return documents


# --- Ingest ---
raw_docs = ingest_documents(DOCS_DIR)
print(f"\n✅ Ingested {len(raw_docs)} documents")
for name, text in raw_docs.items():
    print(f"   • {name}: {len(text):,} chars")

## 2. Text Cleaning

In [ ]:
def clean_text(text: str) -> str:
    """Clean a document's text."""
    # Remove null bytes & non-printable chars (keep newlines/tabs)
    text = re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f\x7f]", "", text)
    # Normalize unicode whitespace
    text = text.replace("\u00a0", " ").replace("\u200b", "")
    # Collapse multiple blank lines into one
    text = re.sub(r"\n{3,}", "\n\n", text)
    # Collapse multiple spaces (but not newlines)
    text = re.sub(r"[^\S\n]+", " ", text)
    # Strip leading/trailing whitespace per line
    text = "\n".join(line.strip() for line in text.splitlines())
    # Final strip
    return text.strip()


cleaned_docs = {name: clean_text(text) for name, text in raw_docs.items()}

print("✅ Cleaning complete")
for name in cleaned_docs:
    before = len(raw_docs[name])
    after = len(cleaned_docs[name])
    diff = before - after
    print(f"   • {name}: {before:,} → {after:,} chars (removed {diff:,})")

## 3. Chunking

In [ ]:
def chunk_text(
    text: str,
    chunk_size: int = CHUNK_SIZE,
    overlap: int = CHUNK_OVERLAP
) -> list[str]:
    """
    Split text into overlapping chunks.
    Tries to break on sentence boundaries when possible.
    """
    if not text:
        return []
    
    chunks = []
    start = 0
    text_len = len(text)
    
    while start < text_len:
        end = start + chunk_size
        
        if end < text_len:
            # Try to find a sentence boundary (., !, ?) near the end
            boundary = -1
            for sep in [". ", "! ", "? ", ".\n", "!\n", "?\n"]:
                pos = text.rfind(sep, start + chunk_size // 2, end)
                if pos > boundary:
                    boundary = pos + 1  # include the punctuation
            if boundary > start:
                end = boundary
        else:
            end = text_len
        
        chunk = text[start:end].strip()
        if chunk:
            chunks.append(chunk)
        
        # Advance with overlap
        start = end - overlap if end < text_len else text_len
    
    return chunks


# Chunk all documents
all_chunks: dict[str, list[str]] = {}
for name, text in cleaned_docs.items():
    chunks = chunk_text(text)
    all_chunks[name] = chunks
    logger.info(f"Chunked {name}: {len(chunks)} chunks")

print(f"\n✅ Chunking complete (chunk_size={CHUNK_SIZE}, overlap={CHUNK_OVERLAP})")
for name, chunks in all_chunks.items():
    print(f"   • {name}: {len(chunks)} chunks")

## 4. Statistics

In [ ]:
flat_chunks = [c for chunks in all_chunks.values() for c in chunks]
chunk_lengths = [len(c) for c in flat_chunks]

stats = {
    "timestamp": datetime.now().isoformat(),
    "config": {"chunk_size": CHUNK_SIZE, "overlap": CHUNK_OVERLAP},
    "total_documents": len(all_chunks),
    "total_chunks": len(flat_chunks),
    "avg_chunk_length": round(sum(chunk_lengths) / len(chunk_lengths), 1) if chunk_lengths else 0,
    "min_chunk_length": min(chunk_lengths, default=0),
    "max_chunk_length": max(chunk_lengths, default=0),
    "chunks_per_document": {
        name: len(chunks) for name, chunks in all_chunks.items()
    }
}

logger.info(f"Total chunks: {stats['total_chunks']}")
logger.info(f"Avg chunk length: {stats['avg_chunk_length']} chars")
logger.info(f"Chunks per doc: {stats['chunks_per_document']}")

# Save stats to JSON
with open("chunk_stats.json", "w") as f:
    json.dump(stats, f, indent=2)

print("\n📊 Pipeline Statistics")
print(f"   Total documents: {stats['total_documents']}")
print(f"   Total chunks:    {stats['total_chunks']}")
print(f"   Avg chunk len:   {stats['avg_chunk_length']} chars")
print(f"   Min chunk len:   {stats['min_chunk_length']} chars")
print(f"   Max chunk len:   {stats['max_chunk_length']} chars")
print(f"\n   Chunks per document:")
for name, count in stats['chunks_per_document'].items():
    print(f"     {name}: {count}")

## 5. Sample Chunks

In [ ]:
import random

NUM_SAMPLES = 5
sample_indices = random.sample(range(len(flat_chunks)), min(NUM_SAMPLES, len(flat_chunks)))

print(f"\n📝 {len(sample_indices)} Sample Chunks\n" + "=" * 60)
for i, idx in enumerate(sample_indices, 1):
    chunk = flat_chunks[idx]
    print(f"\n--- Sample {i} (index={idx}, length={len(chunk)} chars) ---")
    print(chunk[:500] + ("..." if len(chunk) > 500 else ""))
    print()

## 6. Next Steps

```bash
# Push to GitHub
git add document_chunking_pipeline.ipynb chunk_stats.json chunking_pipeline.log
git commit -m "feat: add document chunking pipeline notebook"
git push origin main
```

### Message for Annie

**Subject:** Chunking pipeline ready for review

Hi Annie,

Here's the GitHub link: `<REPO_URL>`

Quick summary:
- **Ingested & cleaned ~10 docs** (txt, pdf, docx, html, etc.) with whitespace normalization, non-printable char removal, and encoding fixes
- **Chunked with size=600 / overlap=100**, using sentence-boundary-aware splitting — stats logged to `chunk_stats.json`
- **No blockers** so far; only dependency note: PDFs need `pymupdf` and DOCX needs `python-docx` (`pip install pymupdf python-docx`)

Let me know if you want to adjust chunk size or add metadata tagging next!